In [1]:
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import InvalidArgumentException
import time
import pandas as pd
import os
from pathlib import Path
import glob
from os.path import join, getsize
import requests

In [2]:
data = pd.read_csv(r'whitelist1.csv')

In [20]:
data.head()

,Name,Symbol,Whitepaper,status,s_name
0,MEDICOHEALTH,MHP,https://medicohealth.io/landing_page/documents...,NaN,medicohealth1
1,ZEUX,ZUC,https://www.zeux.tech/assets/PDF/ZEUX-White-pa...,NaN,zeux
2,BITCADEMY FOOTBALL (PREICO),BTMG,https://bitcademy.io/files/Bitcademy_Whitepape...,NaN,bitcademy_football_(preico)
3,FILMBUSINESSBUSTER,FBB,https://estudios-amazonia.com/FilmBusinessBust...,NaN,filmbusinessbuster
4,GRAYLL (PREICO),GRX,https://grayll.io/wp-content/uploads/2019/06/G...,NaN,grayll_(preico)


In [10]:
sample6k = data[:]
sample6k.shape

(7745, 5)

In [4]:
url_list = sample6k['Whitepaper'].values
name_list = sample6k['s_name'].values

In [5]:
pdf_status = []

for pdf_url, name in zip(url_list, name_list):
    
    def getdirsize(dir):
        size = 0
        for root, dirs, files in os.walk(dir):
            size += sum([getsize(join(root, name)) for name in files])
        return size
    
    before_size = getdirsize(r"C:\Users\higgw\pdf_ico1")
    
    options = webdriver.ChromeOptions()
    options.add_experimental_option("prefs", {
      "download.default_directory": r"C:\Users\higgw\pdf_ico1",
      "download.prompt_for_download": False,
      "download.directory_upgrade": True,
      "safebrowsing.enabled": True,
      "plugins.always_open_pdf_externally": True
    })
    driver = webdriver.Chrome(options=options)
    

    try:
        driver.get(pdf_url)
        #element = WebDriverWait(driver, 12) until(           
        #EC.presence_of_element_located((By.ID, "content")
    
    except (TimeoutException, InvalidArgumentException):      
        pdf_status.append('timeout')
        driver.quit()

    else:
        try:
            time.sleep(8)

            after_size = getdirsize(r"C:\Users\higgw\pdf_ico1")

            list_of_files = glob.glob(r'C:\Users\higgw\pdf_ico1\*')
            latest_file = max(list_of_files, key=os.path.getctime)
            driver.quit()

            if before_size < after_size: 
                list_of_files = glob.glob(r'C:\Users\higgw\pdf_ico1\*')
                latest_file = max(list_of_files, key=os.path.getctime)        
                os.rename(latest_file, r'C:\Users\higgw\pdf_ico1' + '\\' + name + '.pdf')
                pdf_status.append('successful')

            else:
                sc = requests.head(pdf_url)
                pdf_status.append(sc.status_code)
                driver.quit()
        except:       
            pdf_status.append('driver failed')
            driver.quit()



In [19]:
#print(os.path.getmtime(r'C:\pdf_ico\whitepaper.pdf'))

1560713613.7894468


In [46]:
import collections

counter=collections.Counter(pdf_status)
print(counter)

Counter({'successful': 149, 'driver failed': 139, 404: 64, 200: 46, 301: 25, 403: 6, 302: 5, 522: 5, 523: 4, 521: 2, 525: 2, 'timeout': 1, 307: 1, 502: 1, 204: 1})


In [40]:
df = pd.DataFrame({'status':pdf_status})
print (df.head())


          status
0     successful
1  driver failed
2     successful
3     successful
4     successful


In [8]:
my_df = pd.DataFrame(pdf_status)

my_df.to_csv('status_from_.csv', index=False, header=False)